<h1>Submitted By:</h1>
<ol>
    <li><p>Name: Farhan Ishrak Tahmid</p><p>NSU ID: 2031458042</p></li>
    <li><p>Name: Farhana Akbar</p><p>NSU ID: 2012482042</p></li>
</ol>

<h1>Question 1</h1>
<h4>Multi Variable Regression</h4>

In [51]:
# loading data
import pandas as pd
response_file="form-data.csv"
dataframe=pd.read_csv(response_file)
print(dataframe.head)


<bound method NDFrame.head of           Timestamp                Email Address  Gender  \
0               NaN                          NaN     NaN   
1   8/24/2023 23:03      dewanaminul12@gmail.com    Male   
2    8/25/2023 3:21      tanzilshahria@gmail.com    Male   
3    8/25/2023 3:27      progga.tasnim@gmail.com  Female   
4   8/26/2023 11:45  saif.ahmed02@northsouth.edu    Male   
..              ...                          ...     ...   
61   9/17/2023 0:45      knightdark760@gmail.com    Male   
62  9/18/2023 23:43            labib77@gmail.com    Male   
63   9/19/2023 1:13            emon331@gmail.com    Male   
64  9/22/2023 15:47  farabisaifullah32@gmail.com    Male   
65  10/5/2023 12:25         subi.freya@gmail.com  Female   

   Cumulative Grade Point Average(CGPA)  \
0                                   NaN   
1                            3 to < 3.5   
2                              3.5 to 4   
3                              3.5 to 4   
4                              3.5

<h3>Data Cleaning and Scaling</h3>

In [52]:
# Dropping unrelated columns
dataframe=dataframe.drop(columns=['Timestamp','Email Address'])
# Detect if there are missing values
missing_data = dataframe.isna()

# Count the sum of missing values in each column
missing_counts = dataframe.isna().sum()
print(missing_counts)

Gender                                                                                                            1
Cumulative Grade Point Average(CGPA)                                                                              1
Total year now since your graduation                                                                              1
Fields of research interest                                                                                       3
Number of publications                                                                                           27
Were you a teaching assistant (TA) or research assistant(RA) during your time in NSU?                             1
Did you participate in any extra curricular activities while you were studying in NSU?                            1
Were you interested in coding?                                                                                    1
Did you participate in any competitive programming?                     

In [53]:
dataframe['Number of publications'].fillna(0,inplace=True)
dataframe['What was the name of the company you worked for right after your graduation? if your answer is not write N/A.'].fillna('N/A',inplace=True)
dataframe['What was your starting salary range?'].fillna(0,inplace=True)
dataframe['What was the name of the University you completed your higher studies from? if answer is no write N/A.'].fillna('N/A',inplace=True)
dataframe['Where did you start working after completing your higher studies?'].fillna('N/A',inplace=True)


# Detect if there are missing values
missing_data = dataframe.isna()

# Count the sum of missing values in each column
missing_counts = dataframe.isna().sum()
print(missing_counts)

Gender                                                                                                           1
Cumulative Grade Point Average(CGPA)                                                                             1
Total year now since your graduation                                                                             1
Fields of research interest                                                                                      3
Number of publications                                                                                           0
Were you a teaching assistant (TA) or research assistant(RA) during your time in NSU?                            1
Did you participate in any extra curricular activities while you were studying in NSU?                           1
Were you interested in coding?                                                                                   1
Did you participate in any competitive programming?                             

In [54]:
# Find duplicate rows
duplicate_rows = dataframe.duplicated()

# Print the rows that are duplicates
duplicate_data = dataframe[duplicate_rows]
print(duplicate_data)

Empty DataFrame
Columns: [Gender, Cumulative Grade Point Average(CGPA), Total year now since your graduation  , Fields of research interest , Number of publications, Were you a teaching assistant (TA) or research assistant(RA) during your time in NSU?, Did you participate in any extra curricular activities while you were studying in NSU?, Were you interested in coding?, Did you participate in any competitive programming? , Programming languages you know, Frameworks you know, Did you start working as a software engineer after graduation?, What was the name of the company you worked for right after your graduation? if your answer is not write N/A., What was your starting salary range?, Did you have any Start-ups after graduation?, Which country did you go to for higher studies? , What was the name of the University you completed your higher studies from? if answer is no write N/A., Where did you start working after completing your higher studies?]
Index: []


In [55]:
# Split the "Cumulative Grade Point Average(CGPA)" column into "Minimum CGPA" and "Maximum CGPA" columns
dataframe[['Minimum CGPA', 'Maximum CGPA']] = dataframe['Cumulative Grade Point Average(CGPA)'].str.split('<', expand=True)
dataframe[['Minimum CGPA', 'Maximum CGPA']] = dataframe['Minimum CGPA'].str.split(' to ', expand=True)

print(dataframe['Minimum CGPA'])
print(dataframe['Maximum CGPA'])

dataframe['What was your starting salary range?'] = dataframe['What was your starting salary range?'].str.replace('BDT', '')
dataframe['What was your starting salary range?'] = dataframe['What was your starting salary range?'].str.replace('> ', '')    

# Split the "Salary" column into "Minimum Salary" and "Maximum Salary" columns
dataframe[['Minimum Salary', 'Maximum Salary']] = dataframe['What was your starting salary range?'].str.split(' - ', expand=True)
print(dataframe['Minimum Salary'])
print(dataframe['Maximum Salary'])

0     NaN
1       3
2     3.5
3     3.5
4     3.5
     ... 
61    3.5
62    3.5
63    3.5
64      3
65    3.5
Name: Minimum CGPA, Length: 66, dtype: object
0     NaN
1        
2       4
3       4
4       4
     ... 
61      4
62      4
63      4
64       
65      4
Name: Maximum CGPA, Length: 66, dtype: object
0      NaN
1      15K
2      25K
3      40K
4      15K
      ... 
61     NaN
62     NaN
63     25K
64     15K
65     25K
Name: Minimum Salary, Length: 66, dtype: object
0      NaN
1      25K
2      40K
3     None
4      25K
      ... 
61     NaN
62     NaN
63     40K
64     25K
65     40K
Name: Maximum Salary, Length: 66, dtype: object


In [56]:
dataframe['Minimum Salary'] = dataframe['Minimum Salary'].str.replace('k', '000')


<h3>Calculate Correlation and Find the Columns</h3>

In [57]:
import pandas as pd
import numpy as np
from scipy import stats

# Function to calculate Cramér's V for categorical columns
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Calculating Cramér's V for each pair of categorical columns
correlation_matrix = pd.DataFrame(index=dataframe.columns, columns=dataframe.columns, dtype=float)

for col1 in dataframe.columns:
    for col2 in dataframe.columns:
        if col1 != col2:  # Skip self-correlation
            corr = cramers_v(dataframe[col1], dataframe[col2])
            correlation_matrix.loc[col1, col2] = corr

# To get the correlation with the "Minimum Salary" column
minimum_salary_corr = correlation_matrix["Minimum Salary"]
print(minimum_salary_corr)

Gender                                                                                                           0.000000
Cumulative Grade Point Average(CGPA)                                                                             0.183357
Total year now since your graduation                                                                             0.330088
Fields of research interest                                                                                      0.062209
Number of publications                                                                                           0.134048
Were you a teaching assistant (TA) or research assistant(RA) during your time in NSU?                            0.304341
Did you participate in any extra curricular activities while you were studying in NSU?                           0.000000
Were you interested in coding?                                                                                   0.000000
Did you participate in a

<h3>Select Columns and Define it in DataFrame</h3>

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

selected_columns=['Maximum CGPA','Total year now since your graduation  ','Programming languages you know','Did you have any Start-ups after graduation?','Did you participate in any competitive programming? ']

dataframe.dropna(subset=selected_columns, inplace=True)
dataframe.dropna(subset=['Minimum Salary'],inplace=True)

dataframe['Minimum Salary'] = dataframe['Minimum Salary'].str.replace('K', '000')

<h3>Encoding Categorical Data</h3>

In [59]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column_name in selected_columns:
    label_encoder = LabelEncoder()
    dataframe[column_name] = label_encoder.fit_transform(dataframe[column_name])
    label_encoders[column_name] = label_encoder

In [60]:
X = dataframe[selected_columns]
y=dataframe['Minimum Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [61]:
# Create and train a linear regression model
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regression_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared (R2) Score:", r2)

Mean Squared Error: 127900692.88227619
R-squared (R2) Score: -0.9547087025404477


In [62]:
# Predict the entry-level salary for a fresh graduate
fresh_graduate_data = pd.DataFrame({'Maximum CGPA': [3.5], 
                                    'Total year now since your graduation  ': [2], 
                                    'Programming languages you know': [31],
                                    'Did you have any Start-ups after graduation?': [0],
                                    'Did you participate in any competitive programming? ': [1]})

predicted_salary = regression_model.predict(fresh_graduate_data)
print("Predicted Entry-Level Salary:", predicted_salary[0])

Predicted Entry-Level Salary: 49748.707289317405


<h1>Question 2</h1>
<h4>Multi Variable Logistic Regression</h4>

In [63]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [64]:
response_file="form-data.csv"
dataframe=pd.read_csv(response_file)
dataframe = dataframe.drop(0)

<h3>Pre processing data</h3>

In [65]:
# Dropping unrelated columns
dataframe=dataframe.drop(columns=['Timestamp','Email Address'])


# Splitting the "Cumulative Grade Point Average(CGPA)" column into "Minimum CGPA" and "Maximum CGPA" columns
dataframe[['Minimum CGPA', 'Maximum CGPA']] = dataframe['Cumulative Grade Point Average(CGPA)'].str.split('<', expand=True)
dataframe[['Minimum CGPA', 'Maximum CGPA']] = dataframe['Minimum CGPA'].str.split(' to ', expand=True)

dataframe=dataframe.drop('Cumulative Grade Point Average(CGPA)',axis=1)

dataframe['Minimum CGPA'] = pd.to_numeric(dataframe['Minimum CGPA'], errors='coerce')
dataframe['Maximum CGPA'] = pd.to_numeric(dataframe['Maximum CGPA'], errors='coerce')
dataframe[['Minimum CGPA','Maximum CGPA']]=dataframe[['Minimum CGPA','Maximum CGPA']].fillna(0)



dataframe['What was your starting salary range?'] = dataframe['What was your starting salary range?'].str.replace('BDT', '')
dataframe['What was your starting salary range?'] = dataframe['What was your starting salary range?'].str.replace('> ', '')

# Splitting the "Salary" column into "Minimum Salary" and "Maximum Salary" columns
dataframe[['Minimum Salary', 'Maximum Salary']] = dataframe['What was your starting salary range?'].str.split(' - ', expand=True)

dataframe=dataframe.drop('What was your starting salary range?',axis=1)

dataframe['Minimum Salary'] = dataframe['Minimum Salary'].str.replace('K', '000')

dataframe['Maximum Salary'] = dataframe['Maximum Salary'].str.replace('K', '000')

dataframe[['Minimum Salary','Maximum Salary']]=dataframe[['Minimum Salary','Maximum Salary']].fillna(0)

# Converting the data type of the salary columns

dataframe[['Minimum Salary','Maximum Salary']] = dataframe[['Minimum Salary','Maximum Salary']].astype(int)

dataframe['Average Salary']=(dataframe['Minimum Salary']+dataframe['Maximum Salary'])/2
dataframe=dataframe.drop(columns=['Minimum Salary','Maximum Salary'])

# Replacing some row data
dataframe['Total year now since your graduation  ']=dataframe['Total year now since your graduation  '].str.replace(' years','')
dataframe['Total year now since your graduation  ']=dataframe['Total year now since your graduation  '].str.replace(' year','')

# Finding categorical columns
categorical_columns = dataframe.select_dtypes(include=['object', 'category']).columns

# Applying Label Encoding to each categorical column
label_encoders = {}
for column in categorical_columns:
    label_encoder = LabelEncoder()
    dataframe[column] = label_encoder.fit_transform(dataframe[column])
    label_encoders[column] = label_encoder

<h3>Finding the Features which are corelated</h3>

In [66]:
#Finding corelation Between Data columns
correlation_matrix = dataframe.corr()
# Setting the Target column
correlation_with_target = correlation_matrix['Did you start working as a software engineer after graduation?'].abs().sort_values(ascending=False)

#Selecting features based on corelation Matrix
N = 4 # 4 feature to find
selected_features = correlation_with_target.index[1:N+1].tolist()

<h3>Splitting Test and Train data</h3>

In [67]:
#Defining X, Y train test dataset

X = dataframe[selected_features]
y = dataframe['Did you start working as a software engineer after graduation?']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<h3>Applying Regression Model</h3>

In [68]:
# Creating and training a logistic regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = logistic_model.predict(X_test)

<h3>Defining Accuracy Model</h3>

In [69]:
# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion)
print("Classification Report:")
print(classification_rep)

# Feature Importance in the model
feature_importance = logistic_model.coef_
print("Feature Importance (Coefficients):")
for feature, importance in zip(selected_features, feature_importance[0]):
    print(f"{feature}: {importance}")

Accuracy: 0.6923076923076923
Confusion Matrix:
[[8 1]
 [3 1]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.89      0.80         9
           1       0.50      0.25      0.33         4

    accuracy                           0.69        13
   macro avg       0.61      0.57      0.57        13
weighted avg       0.66      0.69      0.66        13

Feature Importance (Coefficients):
Were you interested in coding?: 0.8855848116644424
Total year now since your graduation  : 0.06497684786924997
What was the name of the company you worked for right after your graduation? if your answer is not write N/A.: -0.03384712565510539
Minimum CGPA: -0.7787018258302327


<h3>Making Predictions</h3>

In [70]:
#Predict data
whether_going_or_not=pd.DataFrame({
        'Were you interested in coding?':[0],
        'Total year now since your graduation  ':[1],
        'What was the name of the company you worked for right after your graduation? if your answer is not write N/A.':[33],
        'Minimum CGPA':[2]
    })

prediction=logistic_model.predict(whether_going_or_not)
if prediction==0:
    print(f"Going to a company to do job is False")
else:
    print(f"Going to a company to do job is True")

Going to a company to do job is False


<h1>Question 3</h1>
<h4>For applying KNN algorithm what will be our target function considering important
features as dependent and independent variabls.</h4>

<p>Here i am taking "Average Salary" as my target function considering this as an dependent variable depending on other independent variables</p>

In [71]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.feature_selection import SelectKBest, f_classif


<h3>Declaring Dependent and Independent Variables</h3>
<h4>Getting The selected Features with Kbest Selector with Target function</h4>

In [72]:

# Dependent Variable
y_classification = dataframe['Average Salary']
# Independent Variables
X_classification = df_except_one_column = dataframe.loc[:, dataframe.columns != "Average Salary"]


selector = SelectKBest(score_func=f_classif, k=5)  # selecting 5 features that are most important to predict
selector.fit(X_classification, y_classification)
selected_features = X_classification.columns[selector.get_support()]
print(selected_features)
X_classification = dataframe[selected_columns]


Index(['Number of publications',
       'Were you a teaching assistant (TA) or research assistant(RA) during your time in NSU?',
       'Did you participate in any competitive programming? ',
       'Programming languages you know',
       'What was the name of the company you worked for right after your graduation? if your answer is not write N/A.'],
      dtype='object')


<h3>Splitting into Test Train Data</h3>

In [73]:
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X_classification, y_classification, test_size=0.2)


<h3>Implementing KNN Classifier</h3>

In [74]:
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train_cls, y_train_cls)
classification_result = knn_classifier.predict(X_test_cls)

<h3>Model Accuracy</h3>

In [75]:
classification_accuracy = accuracy_score(y_test_cls, classification_result)
print("Classification Accuracy:", classification_accuracy)



Classification Accuracy: 0.5384615384615384
